### RunnablePassthrough


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
RunnablePassthrough().invoke("Hello, world!")

'Hello, world!'

In [4]:
chat_template_tools = ChatPromptTemplate.from_template(
    """What are the five most importan tools a {job title} needs?
Answer only by listing the tools,
"""
)

chat_template_strategy = ChatPromptTemplate.from_template(
    """Consider the tools provided, develop a strategy for a {job title} to use these tools effectively.
    {tools}
"""
)

In [5]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='What are the five most importan tools a {job title} needs?\nAnswer only by listing the tools,\n'), additional_kwargs={})])

In [6]:
chat = ChatOpenAI(
    api_key=api_key,
    model_name="gpt-4.1-mini",
    temperature=0,
    max_tokens=100,
    seed=42,
)

In [7]:
string_parser = StrOutputParser()

In [ ]:
chain_tools = (
    chat_template_tools | chat | string_parser | {"tools": RunnablePassthrough()}
)
chain_strategy = chat_template_strategy | chat | string_parser

In [9]:
print(chain_tools.invoke({"job title": "data scientist"}))

{'tools': '1. Python  \n2. R  \n3. SQL  \n4. Jupyter Notebook  \n5. Tableau'}


In [10]:
print(
    chain_strategy.invoke(
        {
            "tools": chain_tools.invoke({"job title": "data scientist"}),
            "job title": "data scientist",
        }
    )
)

To effectively leverage the provided tools — Python, R, SQL, Jupyter Notebook, and Tableau — a data scientist can adopt the following strategy:

### 1. Data Extraction and Management (SQL)
- **Use SQL** to query and extract data from relational databases efficiently.
- Write optimized queries to filter, aggregate, and join data as needed.
- Ensure data integrity and perform initial data cleaning at the database level when possible.

### 2. Data Exploration and Cleaning (Python & R)
-


In [11]:
chain_combined = RunnablePassthrough.assign(tools=chain_tools) | chain_strategy

In [12]:
print(chain_combined.invoke({"job title": "data scientist"}))

To use the provided tools effectively, a data scientist can follow this strategic approach:

1. **Data Extraction and Management (SQL)**
   - Use SQL to query and extract data from relational databases.
   - Perform data cleaning and initial transformations directly in the database to reduce data volume and improve efficiency.
   - Write optimized queries to retrieve only the necessary data for analysis.

2. **Data Analysis and Modeling (Python and R)**
   - Use Python for general-purpose programming, data manipulation (with


In [13]:
chain_long = (
    RunnablePassthrough.assign(tools=chat_template_tools | chat | string_parser)
    | chat_template_strategy
    | chat
    | string_parser
)

In [14]:
print(chain_long.invoke({"job title": "data scientist"}))

Certainly! Here's a strategic approach for a data scientist to effectively leverage the provided tools — Python, R, SQL, Jupyter Notebook, and Tableau — throughout the data science workflow:

---

### 1. **Data Acquisition & Storage: SQL**
- **Role:** Extract, manipulate, and manage data stored in relational databases.
- **Strategy:**
  - Use **SQL** to query large datasets efficiently.
  - Perform data cleaning and aggregation at the database level to reduce data transfer and preprocessing time


In [15]:
chain_long.get_graph().print_ascii()

                 +----------------------+               
                 | Parallel<tools>Input |               
                 +----------------------+               
                   ***               ***                
                ***                     ***             
              **                           ***          
+--------------------+                        **        
| ChatPromptTemplate |                         *        
+--------------------+                         *        
           *                                   *        
           *                                   *        
           *                                   *        
    +------------+                             *        
    | ChatOpenAI |                             *        
    +------------+                             *        
           *                                   *        
           *                                   *        
           *                   